# Workshop 6

# Modello geometrico di S. Stefano Rotondo al Celio 

# Modello dela Gerusalemme Celeste

In [1]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.007116 seconds


In [2]:
from larlib import *

##  Misure de riferimento-- (cubito = 0.462 m)

### 1 cubito = 444mm

***
    %-- 1 cubito = 444 mm ----------------------------------------%
    DEF r0 = 24; %---cubiti; raggio interno del muro interno------%
    DEF r2 = 48; %---cubiti; raggio esterno del muro medio--------%
    DEF r3 = 72; %---cubiti; raggio esterno del muro esterno------%
    DEF r4 = 88; %---cubiti; raggio esterno scale esterne---------%

    DEF wsteps = 10;  %---cubiti; profondita' scale esterne-------%
    DEF nsteps = 10;  %---numero alzate/pedate scale esterne------%
    DEF wstep = wsteps/nsteps; %---cubiti; larghezza pedata-------%
    DEF hstep = 1/3; %---cubiti; altezza alzata-------------------%
    DEF hw4 = 29; %--cubiti; altezza all'imposta del muro esterno-%
    DEF hbasament = hstep*nsteps;
***

In [3]:
r0 = 24.0

In [4]:
r2 = 48.0

In [5]:
r3 = 72.0

In [6]:
r4 = 88.0

In [7]:
wsteps = 10.0

In [8]:
nsteps = 10.0

In [9]:
wstep = wsteps/nsteps

In [10]:
hstep = 1.0/3.0

In [11]:
hw4 = 29.0

In [12]:
hbasament = (hstep*nsteps)*1.0

## Mappa cilindrica

***
    DEF cylMap = MAP:[S2 * sin ~ S1, S2 * cos ~ S1, S3];
***

In [13]:
def cylMap():
   
    return MAP([lambda s2: s2[1]*math.sin(s2[0]), 
                lambda s2: s2[1]*math.cos(s2[0]),
                lambda s3: s3[2]])


## Scala esterna

***
    DEF vdom (h::IsRealPos) = ((EMBED:1 ~ Intervals:(PI*3/24)):3) * QUOTE:<h>;
    DEF hdom (w::IsRealPos) = (S:2:-1 ~ EMBED:1):(Intervals:(PI*3/24):3 * QUOTE:<w>);

    DEF steps (w,h::IsRealPos) (n::IsIntPos) = (STRUCT ~ CAT ~ #:n):
      < vdom:h, T:3:h, hdom:w, T:2:(-:w) >;

    DEF hbasament = hstep*nsteps;
    DEF stair = (MKPOL ~ UKPOL):(steps:<wstep,hstep>:nsteps  
                  RIGHT (S:2:-1 ~ @2 ~ CUBOID):<PI/24,wsteps,hbasament>);
    DEF ramp  = cylMap:(T:2:r4:stair);
    DEF stairs = (STRUCT ~ ##:12):<ramp, R:<1,2>:(PI/6)>;
***

In [14]:
def vdom(h):
    return PROD([COMP([EMBED(1),INTERVALS(PI*3.0/24.0)])(3),QUOTE([h])])

In [15]:
def hdom(w):
    return COMP([S(2)(-1),EMBED(1)])(PROD([INTERVALS(PI*3.0/24.0)(3),QUOTE([w])]))

In [16]:
def steps (w,h):
    def steps1(n):
        return COMP([STRUCT,CAT,N(n)])([vdom(h),T(3)(h),hdom(w),T(2)(-w)])
    return steps1

In [17]:
hbasament = hstep*nsteps

In [18]:
stair = RIGHT ([ COMP([MKPOL,UKPOL])(steps(wstep,hstep)(nsteps)),
                COMP([S(2)(-1),SKEL_2,CUBOID])([PI/24.0,wsteps,hbasament]) ])

In [19]:
ramp = cylMap()(T(2)(r4)(stair))

In [20]:
stairs = STRUCT(NN(12)([ramp,R([1,2])(PI/6.0)]))

In [21]:
VIEW(stairs)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1818cdf3f0> >

### Image Stairs

![](img/stairs.png)

## BASAMENTO


***
    DEF basisSector = (cylMap ~ EMBED:1):
        (Intervals:(2*PI/12):4 * Intervals:(r4 - (wstep*(nsteps - 1))):1);
    DEF basis = (STRUCT ~ ##:12):<basisSector, R:<1,2>:(2*PI/12)>;
    DEF basement = (R:<1,2>:(PI/-48) ~ STRUCT):< stairs, T:3:(hstep*nsteps), basis >;
***

In [22]:
def basisSector():
    return COMP([cylMap(),EMBED(1)])(PROD([INTERVALS(2*PI/12)(4),
                                           INTERVALS(r4-(wstep*(nsteps-1)))(1)]))

In [23]:
def basis():
    return COMP([STRUCT,NN(12)])([basisSector(),
                                  R([1,2])(2*PI/12)])

In [24]:
def basement():
    return COMP([R([1,2])(PI/-48),
                 STRUCT])([stairs,T(3)(hstep*nsteps),basis()])

In [25]:
VIEW(basement())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1818cdfd80> >

### Image basamento

![](img/basamento.png)

## Muro externo

***
    DEF ExtWall2Da = MKPOL:<<<0,0>,<7,0>,<7,5>,<0,8>,<7,3>,<9,4>,
    <10.5,1.5>,<10.5,0>,<11,1.5>,<11,3>>,<1..4,<2,8,7,6,5>,<6,7,9,10>>,
    <<1>,<2>,<3>>> & ((Q ~ #:12):1 * Q:8);
    DEF ExtWall2Db = MKPOL:<<<0.5,0>,<4,0>,<5,0>,<6,0>,<6,3>,<5.5,3.5>,
      <5,3>,<4.5,3.5>,<4,3>,<2,4>,<0,3>,<0,1.5>,<0.5,1.5>>,
      <<10,11,12,13>,<1,2,9,10,13>,<2,3,7,8,9>,<3,4,5,6,7>>,<1..4>> 
    &  ((Q ~ #:6):1 * Q:4);
    DEF ExtWall2D = STRUCT:<ExtWall2Da, T:1:11, ExtWall2Db>;
    DEF sizxExtWall = SIZE:1:ExtWall2D;

    DEF ExtWall = R:<2,3>:(PI/2):(ExtWall2D * QUOTE:<1.5>);
    DEF CurvedExtWall = (cylMap ~ T:2:r3 ~ S:<1,3>:<PI/(4*sizxExtWall),hw4/8>):   ExtWall;
    DEF DoubleExtWall = STRUCT:<CurvedExtWall, S:1:-1, CurvedExtWall>;
    DEF FullExtWall = (STRUCT ~ ##:4):<DoubleExtWall, R:<1,2>:(PI/2)>;
***

***
    STRUCT:<basement, T:3:hbasament, FullExtWall>;
***

In [26]:
def ExtWall2Da():
    return INTERSECTION([
    MKPOL([[[0,0],[7,0],[7,5],[0,8],[7,3],[9,4],
        [10.5,1.5],[10.5,0],[11,1.5],[11,3]],
        [range(1,5),[2,8,7,6,5],[6,7,9,10]],[[1],[2],[3]]]),
    PROD([COMP([QUOTE,N(12)])(1),Q(8)])])

In [27]:
def ExtWall2Db():
    return INTERSECTION([
        MKPOL([[[0.5,0],[4,0],[5,0],[6,0],[6,3],[5.5,3.5],
            [5,3],[4.5,3.5],[4,3],[2,4],[0,3],[0,1.5],[0.5,1.5]],
            [[10,11,12,13],[1,2,9,10,13],[2,3,7,8,9],[3,4,5,6,7]],
            range(1,5)]),
        PROD([COMP([QUOTE,N(6)])(1),Q(4)])
    ])

In [28]:
def ExtWall2D():
    return STRUCT([ExtWall2Da(),
                   T(1)(11),ExtWall2Db()])

In [29]:
def sizxExtWall():
    return SIZE(1)(ExtWall2D())

In [30]:
def ExtWall():
    return R([2,3])(PI/2)(PROD([ExtWall2D(),
                                QUOTE([1,5])]))

In [31]:
def CurvedExtWall():
    return COMP([cylMap(),T(2)(r3),
                 S([1,3])([PI/(4*sizxExtWall()),hw4/8])])(ExtWall())

In [32]:
def DoubleExtWall():
    return STRUCT([CurvedExtWall(),S(1)(-1),
                   CurvedExtWall()])

In [33]:
def FullExtWall():
    return COMP([STRUCT,NN(4)])([DoubleExtWall(),R([1,2])(PI/2)])

In [34]:
VIEW(FullExtWall())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1818ccfe70> >

### Image FullExtWall

![](img/Full_Ext_wall.png)

In [35]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f202c00> >

### Image aggr1

![](img/aggr1.png)

## Colonne intermedie

***
    DEF Column (w,h::IsRealPos) = basis TOP trunk TOP capitel
    WHERE
      basis = CUBOID:<w,w,2*w/3>,
      trunk = CYLINDER:<w/2*0.85, h - w>:8,
      capitel = CUBOID:<w,w,w/3>
    END;
    DEF arcAngle = 2*PI/50.4;
    DEF wallAngle = -3.2*arcAngle/4;
    DEF RotCross = 3.2*arcAngle/4 + 2.5*arcAngle;

    DEF MyColumn = (MKPOL ~ UKPOL ~ T:2:((r2 - 0.75)) ~ Column):<1.5,hCol>;
    DEF The4cols = (R:<1,2>:(arcAngle*0.4/3.2) ~ MKPOL ~ UKPOL ~ 
      STRUCT ~ ##:4):< R:<1,2>:(-:arcAngle), MyColumn >;
    DEF The5cols = (R:<1,2>:(arcAngle*0.4/3.2) ~ MKPOL ~ UKPOL ~ 
      STRUCT ~ ##:5):< R:<1,2>:(-:arcAngle), MyColumn >;
    DEF TheBotWal = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ CUBOID):
      < 3.2*arcAngle/4, 1.5, hCol >;
    DEF TheSecCols = STRUCT:< R:<1,2>:RotCross,
      TheBotWal, R:<1,2>:wallAngle, 
      The4cols, R:<1,2>:(-5*arcAngle), 
      TheBotWal, R:<1,2>:wallAngle, 
      The5cols >;
    DEF TheMedColumns = (STRUCT ~ ##:4):<TheSecCols, R:<1,2>:(PI/-2)>;
    DEF hCol = 12 ;
***

***
    STRUCT:<basement, T:3:hbasament, FullExtWall, TheMedColumns >;
***

In [36]:
def Column(argomento):
    w,h = argomento
    basis = CUBOID([w,w,2*w/3])
    trunk = CYLINDER([w/2*0.85,h-w])(8)
    capitel = CUBOID([w,w,w/3])
    return TOP([TOP([basis,trunk]),capitel])

In [37]:
hCol = 12

In [38]:
arcAngle = 2*PI/50.4

In [39]:
wallAngle = -3.2*arcAngle/4

In [40]:
RotCross = 3.2*arcAngle/4

In [41]:
def MyColumn():
    return COMP([MKPOL,UKPOL,T(2)(r2-0.75),Column])([1.5,hCol])

In [42]:
def The4cols():
    return COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(4)])([R([1,2])(-arcAngle),MyColumn()])

In [43]:
def The5Cols():
    return COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(5)])([R([1,2])(-arcAngle),MyColumn()])

In [44]:
def TheBotWal():
    return COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),CUBOID]) ([3.2*arcAngle/4,1.5,hCol])

In [45]:
def TheSecCols():
    return STRUCT([R([1,2])(RotCross),
                  TheBotWal(),R([1,2])(wallAngle),
                  The4cols(),R([1,2])(-5*arcAngle),
                  TheBotWal(),R([1,2])(wallAngle),
                  The5Cols()] )

In [46]:
def TheMedColumns():
    return COMP([STRUCT,NN(4)])([TheSecCols(),R([1,2])(PI/(-2))])

In [47]:
VIEW(TheMedColumns())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1818d03e40> >

### Image TheMedColumns

![](img/TheMedColumns.png)

In [48]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),TheMedColumns()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1818d03f90> >

### Image aggr2

![](img/aggr2.png)

## Archi sulle colonne

***
    DEF bottomArc (d::IsRealPos) = Bezier:S1:<<0,0>,<0,2*d/3>,<d,2*d/3>,<d,0>>;
    DEF topArc (d::IsRealPos) = Bezier:S1:<<0,2*d/3>,<d,2*d/3>>;
    DEF arc2D  (d::IsRealPos) = Bezier:S2:<bottomArc:d,topArc:d>;
    DEF arc3D  (d::IsRealPos)(w::IsRealPos) = (T:2:w ~ R:<2,3>:(PI/2)):
      ( MAP:(CONS:(arc2D:d)):(Intervals:1:8 * Intervals:1:1) * QUOTE:<w> );
    DEF Interarc (d1,d2::IsReal)(w::IsRealPos)= CUBOID:<d1,w,2*d2/3>;
    DEF Xarc (d1,d2::IsRealPos) (w::IsRealPos) = 
      Interarc:<d1,d2>:w RIGHT arc3D:d2:w RIGHT Interarc:<d1,d2>:w;

    DEF SizeArc = SIZE:1:TheArc;
    DEF TheArc = Xarc:<0.4,3.2>:1.5;
    DEF HeigthArc = SIZE:3:CurvedArc;
    DEF CurvedArc = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ S:<1,3>:<sx,sz>):TheArc
    WHERE
      sx = (1/SizeArc)*arcAngle,
      sz = sx * (r2 - 2)
    END;
    DEF The5arcs = (MKPOL ~ UKPOL ~ STRUCT ~ ##:5):< CurvedArc, R:<1,2>:(-:arcAngle) >;
    DEF The6arcs = (MKPOL ~ UKPOL ~ STRUCT ~ ##:6):< CurvedArc, R:<1,2>:(-:arcAngle) >;
    DEF TheMidWall = (cylMap ~ MKPOL ~ UKPOL ~ T:2:(r2 - 0.75) ~ CUBOID):
      <3.2*arcAngle/4, 1.5, HeigthArc>;
    DEF TheSector = STRUCT:< R:<1,2>:RotCross,
      TheMidWall, R:<1,2>:wallAngle, 
      The5arcs, R:<1,2>:(-5*arcAngle), 
      TheMidWall, R:<1,2>:wallAngle, 
      The6arcs >;
    DEF TheArcs = (STRUCT ~ ##:4):<TheSector, R:<1,2>:(PI/-2)>;
    DEF TheArcsCols = STRUCT:< TheMedColumns, T:3:hCol, TheArcs >;
***

***
    STRUCT:<basement, T:3:hbasament, FullExtWall, TheArcsCols >;
***

In [49]:
def bottomArc(d):
    return BEZIER(S1)([[0,0],[0,2*d/3],[d,2*d/3],[d,0]])

In [50]:
def topArc(d):
    return BEZIER(S1)([[0,2*d/3],[d,2*d/3]])

In [51]:
def arc2D(d):
    return BEZIER(S2)([bottomArc(d),topArc(d)])

In [52]:
def arc3D(d):
    def arc3D1(w):
        return COMP([T(2)(w),R([2,3])(PI/2)])(MAP(arc2D(d))(PROD([PROD([INTERVALS(1)(8),
                                                                        INTERVALS(1)(1)]),QUOTE([w])])))
    return arc3D1

In [53]:
def Interarc(d1,d2):
    def Interarc1(w):
        return CUBOID([d1,w,2*d2/3])
    return Interarc1

In [54]:
def Xarc(d1,d2):
    def Xarc1(w):
        return RIGHT([RIGHT([Interarc(d1,d2)(w),arc3D(d2)(w)]),Interarc(d1,d2)(w)])
    return Xarc1

In [55]:
TheArc = Xarc(0.4,3.2)(1.5)

In [56]:
SizeArc = SIZE(1)(TheArc)

In [57]:
def CurvedArc():
    sx = (1/SizeArc)*arcAngle
    sz = sx *(r2-2)
    return COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),S([1,3])([sx,sz])])(TheArc)

In [58]:
HeightArc = SIZE(3)(CurvedArc())

In [59]:
def The5arcs():
    return COMP([MKPOL,UKPOL,STRUCT,NN(5)])([CurvedArc(),R([1,2])(-arcAngle)])
    

In [60]:
def The6arcs():
    return COMP([MKPOL,UKPOL,STRUCT,NN(6)])([CurvedArc(),R([1,2])(-arcAngle)])

In [61]:
def TheMidWall():
    return COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4.0,1.5,HeightArc])

In [62]:
def TheSector():
    return STRUCT([R([1,2])(RotCross),TheMidWall(),
                   R([1,2])(wallAngle),The5arcs(),R([1,2])(-5*arcAngle),
                   TheMidWall(),R([1,2])(wallAngle),The6arcs()])

In [63]:
def TheArcs():
    return COMP([STRUCT,NN(4)])([TheSector(),R([1,2])(PI/-2)])

In [64]:
def TheArcsCols():
    return STRUCT([TheMedColumns(),T(3)(hCol),TheArcs()])

In [65]:
VIEW(TheArcsCols())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f213750> >

### Image aggr3

![](img/aggr3.png)

In [66]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),TheArcsCols()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f213780> >

![](img/Struct_TheArcsCols.png)

## Muro intermedio superiore

***
    DEF MedWall2D = MKPOL:<<<0,4>,<8,4>,<17,4>,<17,5>,<12.5,7.5>,<8,5>,<0,8>>,
    <<1,2,6,7>,<2,3,4,5,6>>,
    <<1>,<2>>> & (QUOTE:(#:17:1) * QUOTE:<8>);
    DEF sizxMedWall = SIZE:1:MedWall2D;

    DEF MedWall = R:<2,3>:(PI/2):(MedWall2D * QUOTE:<1.5>);
    DEF CurvedMedWall = (cylMap ~ MKPOL ~ UKPOL ~ 
        T:2:(r2+0.75) ~ S:<1,3>:<PI/(4*sizxMedWall),hw4/8>):MedWall;

    DEF DoubleMedWall = STRUCT:<CurvedMedWall, S:1:-1:CurvedMedWall>;
    DEF FullMedWall = (STRUCT ~ ##:4):<DoubleMedWall, R:<1,2>:(PI/2)>;
    DEF MedWallArcs = STRUCT:< FullMedWall, TheArcsCols >;
***                   

***
    STRUCT:<basement, T:3:hbasament, FullExtWall, MedWallArcs >;
***

In [67]:
def MedWall2D():
    return INTERSECTION([MKPOL([[[0,4],[8,4],[17,4],[17,5],[12.5,7.5],[8,5],[0,8]],
                               [[1,2,6,7],[2,3,4,5,6]],[[1],[2]]]),
                       PROD([COMP([QUOTE,N(17)])(1),QUOTE([8])])])

In [68]:
sizxMedWall = SIZE(1)(MedWall2D())

In [69]:
MedWall = R([2,3])(PI/2)(PROD([MedWall2D(),QUOTE([1.5])]))

In [70]:
def CurvedMedWall():
    return COMP([cylMap(),MKPOL,UKPOL,T(2)(r2+0.75),
                 S([1,3])([PI/(4*sizxMedWall),hw4/8])])(MedWall)

In [71]:
def DoubleMedWall():
    return STRUCT([CurvedMedWall(),S(1)(-1)(CurvedMedWall())])

In [72]:
def FullMedWall():
    return COMP([STRUCT,NN(4)])([DoubleMedWall(),R([1,2])(PI/2)])

In [73]:
def MedWallArcs():
    return STRUCT([FullMedWall(),TheArcsCols()])

In [74]:
VIEW(MedWallArcs())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f20f2d0> >

### Image MedWallArcs

![](img/MedWallArcs.png)

In [75]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),MedWallArcs()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f20f150> >

![](img/struct_MedWallArcs.png)

## Muri laterali della croce

***
    DEF CrossWall = (T:1:-1 ~ R:<1,2>:(0.2*arcAngle) ~ 
      S:3:(hw4/8) ~ CUBOID):<1.5,r3 - r2,5>;
    DEF SingleWall =  (R:<1,2>:(2.5*arcAngle) ~ T:2:r2):CrossWall;
    DEF DoubleWall =  STRUCT:< SingleWall, S:1:-1, SingleWall >;
    DEF CrossWalls = (STRUCT ~ ##:4):<DoubleWall, R:<1,2>:(PI/2)>;
***                      

***
    STRUCT:<basement, T:3:hbasament, FullExtWall, MedWallArcs,   CrossWalls >;
***

In [76]:
def CrossWall():
    return COMP([T(1)(-1),R([1,2])(0.2*arcAngle),S(3)(hw4/8),CUBOID])([1.5,r3-r2,5])

In [77]:
def SingleWall():
    return COMP([R([1,2])(2.5*arcAngle),T(2)(r2)])(CrossWall())

In [78]:
def DoubleWall():
    return STRUCT([SingleWall(),S(1)(-1),SingleWall()])

In [79]:
def CrossWalls():
    return COMP([STRUCT,NN(4)])([DoubleWall(),R([1,2])(PI/2)])

In [80]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),MedWallArcs(),CrossWalls()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f20f930> >

![](img/muri_laterali.png)

## Tetto sulla cruce

***
    DEF HalfTetto = MKPOL:<<<0,r2,8>,<8,r2,5>,<8,r3,5>,<0,r3,8>>,<1..4>,<<1>>>;
    DEF CurvedHalfTetto = (cylMap ~ S:<1,3>:<2.5*arcAngle/8,hw4/8>):HalfTetto;
    DEF DoubleHalfTetto = STRUCT:<CurvedHalfTetto, S:1:-1:CurvedHalfTetto>;
    DEF FullTetto = (STRUCT ~ ##:4):<DoubleHalfTetto, R:<1,2>:(PI/-2)>;
***

***
    STRUCT:<basement, T:3:hbasament, FullExtWall, MedWallArcs, 
        CrossWalls, FullTetto >;
***        

In [81]:
def HalfTetto():
    return MKPOL([[[0,r2,8],[8,r2,5],[8,r3,5],[0,r3,8]],[[1,2,3,4]],[]])

In [82]:
def CurvedHalfTetto():
    return COMP([cylMap(),S([1,3])([2.5*arcAngle/8,hw4/8])])(HalfTetto())

In [83]:
def DoubleHalfTetto():
    return STRUCT([CurvedHalfTetto(),S(1)(-1)(CurvedHalfTetto())])

In [84]:
def FullTetto():
    return COMP([STRUCT,NN(4)])([DoubleHalfTetto(),R([1,2])(PI/-2)])

In [85]:
VIEW(STRUCT([CrossWalls(),FullTetto()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f20ff00> >

### Image crosswalls

![](img/crosswalls.png)

In [86]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),
             MedWallArcs(),CrossWalls(),FullTetto()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f20fde0> >

### Image aggr4

![](img/aggr4.png)

## Muro con Triplice apertura

***
    DEF CourtWall (d1,d2::IsRealPos) (n1,n2::IsIntPos) = 
      LeftWall OP TripleHole OP RightWall
    WHERE
      op = ALIGN:<<1,MAX,MIN>,<2,MIN,MIN>>,
      LeftWall = ((QUOTE ~ #:n1):(d1/n1)) * CUBOID:<1,h>,
      RightWall = ((QUOTE ~ #:n2):(d2/n2)) * CUBOID:<1,h>,
      h = SIZE:3:TripleHole,
      TripleHole = STRUCT:<column:<w,2>, T:1:(2+w), column:<w,2>> 
        TOP Xarc:<2,2>:1,
      w = 0.7
    END;
***

***
    CourtWall:<6,6>:<3,3>
***

In [87]:
def CourtWall(d1,d2):
   
    def CourtWall_0(n1,n2):
        op = ALIGN([[1,MAX,MIN],[2,MIN,MIN]])
        w=0.7
        TripleHole = TOP([STRUCT([Column([w,2]),
                                  T(1)(2+w),Column([w,2])]),Xarc(2,2)(1)])
        h=SIZE(3)(TripleHole)
        
        LeftWall = PROD([COMP([QUOTE,N(n1)])
                               (d1/n1),CUBOID([1,h])])
        RightWall =PROD([COMP([QUOTE,N(n2)])(d2/n2),CUBOID([1,h])])
        
        
        
        return op ([op([LeftWall,TripleHole]),RightWall])
    return CourtWall_0

In [88]:
view_CourtWall = CourtWall(6,6)(3,3)

In [89]:
VIEW(view_CourtWall)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f200840> >

### Image CourtWall

![](img/CourtWall.png)

## Muro inferiore cortile interno

***
    DEF MyCourtWall = CourtWall:<12,12>:<9,9>;
    DEF sizxCourtWall = SIZE:1:MyCourtWall;
    DEF CurvedCourtWall = (optimize ~ 
        R:<1,2>:(-2.5*arcAngle) ~ mapping ~ T:2:((r3+r2)/2)): MyCourtWall;
    DEF mapping = (cylMap ~ S:<1,3>:<(7.25*PI/24)*(1/sizxCourtWall),hw4/8>);
    DEF CrossCourtWall = (STRUCT ~ ##:4):<CurvedCourtWall, R:<1,2>:(PI/2)>;
    %
    STRUCT:<FullTetto, CrossWalls, CrossCourtWall >;
    STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns>;
***

In [90]:
MyCourtWall = CourtWall(12,12)(9,9)

In [91]:
sizxCourtWall = SIZE(1)(MyCourtWall)

In [92]:
def mapping():
    return COMP([cylMap(),S([1,3])([(7.25*PI/24)*(1/sizxCourtWall),hw4/8])])

In [93]:
def CurvedCourtWall():
    return COMP([OPTIMIZE,R([1,2])(-2.5*arcAngle),mapping(),T(2)((r3+r2)/2)])(MyCourtWall)

In [94]:
def CrossCourtWall():
    return COMP([STRUCT,NN(4)])([CurvedCourtWall(),R([1,2])(PI/2)])

In [95]:
VIEW(CrossCourtWall())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f200db0> >

### Image CrossCourtWall

![](img/CrossCourtWall.png)

In [96]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),
             MedWallArcs(),CrossWalls(),FullTetto(),CrossCourtWall()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f22e0c0> >

![](img/struct_CrossCourtWall.png)

## Coronamento muro Cortile interno

***
    DEF TopCourtWall2D = MKPOL:<<<0,3.333>,<10,3.333>,<10,4.333>,<5,6.333>,<0,4.333>>,
        <1..5>,<<1>>> & (QUOTE:(#:10:1) * QUOTE:<8>);
    DEF TopCourtWall = R:<2,3>:(PI/2):(TopCourtWall2D * QUOTE:<1>);
    DEF MyTopCourtWall = TopCourtWall RIGHT TopCourtWall RIGHT TopCourtWall; 
    DEF CurvedTopCourtWall = (
        R:<1,2>:(-2.5*arcAngle) ~ mapping ~ T:2:((r3+r2+2)/2)): MyTopCourtWall;
    DEF FullTopCourtWall = (STRUCT ~ ##:4):<CurvedTopCourtWall, R:<1,2>:(PI/2)>;
***

***
    STRUCT:<FullTetto, CrossWalls, CrossCourtWall, FullTopCourtWall >;
***

***
    STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
        CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall>;
***        


In [97]:
def TopCourtWall2D():
    return INTERSECTION([
        MKPOL([[[0,3.333],
        [10,3.333],[10,4.333],
        [5,6.333],[0,4.333]],[range(1,6)],[[1]]]),
        PROD([COMP([QUOTE,N(10)])(1),QUOTE([8])])])
                                 
    

In [98]:
def TopCourtWall():
    return R([2,3])(PI/2)(PROD([TopCourtWall2D(),QUOTE([1])]))

In [99]:
MyTopCourtWall = RIGHT([RIGHT([TopCourtWall(),TopCourtWall()]),TopCourtWall()])

In [100]:
def CurvedTopCourtWall():
    return COMP([R([1,2])(-2.5*arcAngle),
            mapping(),T(2)((r3+r2+2)/2)])(MyTopCourtWall)

In [101]:
def FullTopCourtWall():
    return COMP([STRUCT,NN(4)])([CurvedTopCourtWall(),
                R([1,2])(PI/2)])

In [102]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),
             MedWallArcs(),CrossWalls(),
             FullTetto(),CrossCourtWall(),
             FullTopCourtWall()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f22e6c0> >

![](img/FullTopCourtWall.png)

## Tetto su Cortile interno

    DEF RotWall = 2.5*arcAngle;
    DEF CourTetto = MKPOL:<<<0,(r3+r2+2)/2,3.333>,<10,(r3+r2+2)/2,3.333>,
        <10,r2,4.333>,<0,r2,4.333>,
        <0,(r3+r2+2)/2,3.433>,<10,(r3+r2+2)/2,3.433>,
        <10,r2,4.433>,<0,r2,4.433> >, <1..8>,<<1>>> 
    & 
       (QUOTE:(#:10:1) * QUOTE:<(r3+r2+2)/2> * QUOTE:<5>);

    DEF MyCourTetto = CourTetto RIGHT CourTetto RIGHT CourTetto; 
    DEF CurvedCourTetto = (R:<1,2>:(-:RotWall) ~ mapping): MyCourTetto;
    DEF FullCourTetto = (STRUCT ~ ##:4):<CurvedCourTetto, R:<1,2>:(PI/2)>;

    %
    STRUCT:<FullTetto, CrossWalls, CrossCourtWall, FullTopCourtWall,
            FullCourTetto >;
    STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, FullCourTetto>;

In [103]:
RotWall = 2.5*arcAngle

In [104]:
def CourTetto():
    return INTERSECTION([
        MKPOL([[[0,(r3+r2+2)/2,3.333],
               [10,(r3+r2+2)/2,3.333],
               [10,r2,4.333],[0,r2,4.333],
               [0,(r3+r2+2)/2,3.433],
               [10,(r3+r2+2)/2,3.433],
               [10,r2,4.433],[0,r2,4.433]],
               [range(1,9)],[[1]]]),PROD([COMP([
            QUOTE,N(10)])(1),PROD([QUOTE([
            (r3+r2+2)/2]),QUOTE([5])])])])
        

In [105]:
MyCourTetto = RIGHT([RIGHT([CourTetto(),CourTetto()]),CourTetto()])

In [106]:
def CurvedCourTetto():
    return COMP([R([1,2])(DIFF(RotWall)),mapping()])(MyCourTetto)

In [107]:
def FullCourTetto():
    return COMP([STRUCT,NN(4)])([CurvedCourTetto(),R([1,2])(PI/2.0)])
    

In [108]:
VIEW(FullCourTetto())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f230540> >

### Image FullCourTetto

![](img/CourTetto.png)

In [109]:
VIEW(STRUCT([MedWallArcs(),FullTopCourtWall(),
             FullCourTetto(),CrossWalls(),
             FullTetto(),CrossCourtWall()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f230750> >

![](img/fullcourtetto.png)

In [110]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),
             MedWallArcs(),CrossWalls(),FullTetto(),
             CrossCourtWall(),FullTopCourtWall(),
             FullCourTetto()]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f230870> >

### Image aggr5

![](img/aggr5.png)

## Colonne muro interno


***    
    DEF Radial22Obj (Obj::IsPol) = 
      STRUCT:< The20Obj, R:<1,2>:((20*PI)/11), The2Obj >
    WHERE
      The4Obj = (STRUCT ~ ##:4):< Obj, R:<1,2>:(PI/11) >,
      The2Obj  = (STRUCT ~ ##:2):< Obj, R:<1,2>:(PI/11) >,
      The20Obj = (STRUCT ~ ##:5):< The4Obj, R:<1,2>:((4*PI)/11) >
    END;

    DEF InternalCol = (MKPOL ~ UKPOL ~ T:2:((r0 - 2)) ~ Column):<1.5,hCol>;
    DEF The22Columns = Radial22Obj:InternalCol;
***

In [111]:
def Radial22Obj(Obj):
    The4Obj = COMP([STRUCT,NN(4)])([Obj,R([1,2])(PI/11)])
    The2Obj = COMP([STRUCT,NN(2)])([Obj,R([1,2])(PI/11)])
    The20Obj = COMP([STRUCT,NN(5)])([The4Obj,R([1,2])((4*PI)/11)])
    return STRUCT([The20Obj,R([1,2])(20*PI/11),The2Obj])

In [112]:
def InternalCol():
    return COMP([MKPOL,UKPOL,T(2)((r0-2)),Column])([1.5,hCol])

In [113]:
The22Columns = Radial22Obj(InternalCol())

## Travi muro interno

***
        DEF Beam = Q:<-0.5,1.5,2,1.5> * Q:<2.2> * Q:1.5;
    DEF CurvedBeam = (cylMap ~ T:<2,3>:<(r0 - 2),hcol> ~ S:1:(PI/(11*5)) ):Beam;
    DEF The22Beams = Radial22Obj:CurvedBeam;
***

In [114]:
def Beam():
    return PROD([QUOTE([-0.5,1.5,2,1.5]),PROD([QUOTE([2.2]),QUOTE([1.5])])])

In [115]:
def CurvedBeam():
    return COMP([cylMap(),T([2,3])([(r0-2),hCol]),S(1)(PI/(11*5))])(Beam())

In [116]:
The22Beams = Radial22Obj(CurvedBeam())

In [117]:
VIEW(STRUCT([The22Columns,The22Beams]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f230ed0> >

### the22Bems

![](img/the22Bems.png)

In [118]:
VIEW(STRUCT([basement(),T(3)(hbasament),
             FullExtWall(),MedWallArcs(),
             CrossWalls(),FullTetto(),
             CrossCourtWall(),FullTopCourtWall(),
             FullCourTetto(),The22Columns,The22Beams]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f23d240> >

### Image aggr6

![](img/aggr6.png)

## Muro interno superiore

***    
        DEF Hwall = STRUCT:<
    %  QUOTE:<-0.5,1.5,-2,1.5> * QUOTE:<-0.3,1.4> * QUOTE:<2.5*hCol>,
      QUOTE:<-2.0,2> * QUOTE:<-0.3,1.4> * QUOTE:<1.5*hCol, -0.5*hcol, 0.5*hcol>
    %  QUOTE:<-0.5,1.75,-1.5,1.75> * QUOTE:<-0.3,1.4> * QUOTE:<2.5*hCol>,
      QUOTE:<-2.25,1.5> * QUOTE:<-0.3,1.4> * QUOTE:<1.5*hCol, -0.5*hcol, 0.5*hcol>
    >;
    DEF CurvedHwall = (cylMap ~ T:<2,3>:<(r0 - 2),hcol> ~ S:1:(PI/(11*5)) ):Hwall;
    DEF The22Hwall = Radial22Obj:CurvedHwall;
    %
    STRUCT:< The22Columns, The22Beams, The22Hwall >;

    STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, 
    CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns,                  FullTopCourtWall, 
            FullCourTetto, The22Columns, The22Beams, The22Hwall >;         
***

In [119]:
def Hwall():
    return STRUCT([INSR(PROD)([QUOTE([-0.5,1.5,-2,1.5]),QUOTE([-0.3,1.4]),QUOTE([2.5*hCol])]),
               INSR(PROD)([QUOTE([-2.0,2]),QUOTE([-0.3,1.4]),QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol])]),
               INSR(PROD)([QUOTE([-0.5,1.75,-1.5,1.75]),QUOTE([-0.3,1.4]),QUOTE([2.5*hCol])]),
               INSR(PROD)([QUOTE([-2.25,1.5]),QUOTE([-0.3,1.4]),QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol])])])

In [120]:
def CurvedHwall():
    return COMP([cylMap(),T([2,3])
                 ([r0-2,hCol]),S(1)(PI/(11*5))])(Hwall())

In [121]:
The22Hwall = Radial22Obj(CurvedHwall())

In [122]:
VIEW(The22Hwall)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f23dc30> >

### Image The22Hwall

![](img/The22Hwall.png)

In [123]:
element = STRUCT([The22Columns,The22Beams,The22Hwall])


In [124]:
VIEW(element)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f23de10> >

![](img/element.png)

In [125]:
VIEW(STRUCT([basement(),T(3)(hbasament),
             FullExtWall(),MedWallArcs(),
             CrossWalls(),FullTetto(),
             CrossCourtWall(),FullTopCourtWall(),
             FullCourTetto(),The22Columns,
             The22Beams,The22Hwall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f218030> >

![](img/struct_The22Hwall.png)

## Coronamento superiore muro interno

***
    DEF myArc = MAP:[SIN ~ S1, COS ~ S1]:(QUOTE:(#:5:(PI/10)));
    DEF Vertex = MKPOL:<<<0,0>>,<<1>>,<<1>>>;
    DEF Sector = JOIN:<myArc, Vertex>;
    DEF Triangle = S:1:-1:(SIMPLEX:2);
    DEF SectorTriangle2D = STRUCT:<T:1:0.05, Sector, T:1:2, Triangle>;
    DEF DoubleSect = STRUCT:< SectorTriangle2D, S:1:-1:SectorTriangle2D>;
    DEF Sect2D = S:1:(2/2.05):DoubleSect;
    DEF Sect3D = R:<2,3>:(PI/2):(Sect2D * QUOTE:<0.8>);
    DEF CurvedSect = (cylMap ~ T:2:(r0 %- 2%) ~ S:<1,3>:<(2*PI)/(11*4),3> ):Sect3D;
    DEF Frieze = T:3:(3.5*hcol):CurvedSect;
    DEF Radial11Obj (Obj::IsPol) = 
      STRUCT:< The10Obj, R:<1,2>:((20*PI)/11), Obj >
    WHERE
      The2Obj = (STRUCT ~ ##:2):< Obj, R:<1,2>:((2*PI)/11) >,
      The10Obj = (STRUCT ~ ##:5):< The2Obj, R:<1,2>:((4*PI)/11) >
    END;
    DEF The11Frieze = Radial11Obj:Frieze;
***

***
    The11Frieze;
    STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
            FullCourTetto, The22Columns, The22Beams, The22Hwall, The11Frieze >;
***

In [126]:
def myArc():
    return MAP([COMP([SIN,S1]),
                COMP([COS,S1])])(COMP([QUOTE,N(5)])(PI/10))

In [127]:
def Vertex():
    return MKPOL([[[0,0]],[[1]],[[1]]])

In [128]:
def Sector():
    return JOIN([myArc(),Vertex()])

In [129]:
Triangle = S(1)(-1)(SIMPLEX(2))

In [130]:
def SectorTriangle2D():
    return STRUCT([T(1)(0.05),Sector(),T(1)(2),Triangle])

In [131]:
def DoubleSect():
    return STRUCT([SectorTriangle2D(),S(1)(-1)(SectorTriangle2D())])

In [132]:
Sect2D = S(1)(2/2.05)(DoubleSect())

In [133]:
Sect3D = R([2,3])(PI/2)(PROD([Sect2D,QUOTE([0.8])]))

In [134]:
def CurvedSect():
    return COMP([cylMap(),T(2)(r0),
                 S([1,3])([(2*PI)/(11*4),3])])(Sect3D)

In [135]:
Frieze = T(3)(3.5*hCol)(CurvedSect())

In [136]:
def Radial11Obj(Obj):
    The2Obj = COMP([STRUCT,NN(2)])([Obj,R([1,2])((2*PI)/11)])
    The10Obj = COMP([STRUCT,NN(5)])([The2Obj,R([1,2])((4*PI)/11)])
    return STRUCT([The10Obj,R([1,2])((20*PI)/11),Obj])

In [137]:
The11Frieze = Radial11Obj(Frieze)

In [138]:
VIEW(The11Frieze)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f218870> >

### Image The11Frieze

![](img/The_11_Frieze.png)

In [139]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),
             MedWallArcs(),CrossWalls(),FullTetto(),
             CrossCourtWall(),FullTopCourtWall(),
             FullCourTetto(),The22Columns,
             The22Beams,The22Hwall,The11Frieze]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f218ea0> >

![](img/struct_the_11_Frieze.png)

## Traliccio Tetto Interno

***
    
    DEF Trellis (H,L::IsRealPos) = (R:<2,3>:(PI/2) ~ EMBED:1 ~ MKPOL):
      <<<0,0>,<L/3,0>,<2*L/3,0>,<L,0>,<2*L/3,H/3>,<L/3,2*H/3>,<0,H>>,
       <<1,2>,<2,3>,<3,4>,<4,5>,<5,6>,<6,7>,<7,1>,<1,6>,<6,2>,<2,5>,<5,3>>,
       <1..11> > ;
    DEF Radial (n::IsIntPos)(obj::IsPol) = (STRUCT ~ ##:3):
       <(STRUCT ~ ##:(n/3)):< obj, R:<1,2>:(2*PI/n) >, R:<1,2>:(2*PI/3)>;

    DEF TrellisTop3D = (T:2:-0.25 ~ S:2:0.5 ~ OffSet:<1,1,1> ~ Trellis):<hcol,r0 - 1>;
    DEF TheTopTrellis = T:3:(3.5*hcol):TrellisTop3D;
    DEF TopTrellis = Radial:12:TheTopTrellis;
***

***
    STRUCT:< The11Frieze, TopTrellis >;

    STRUCT:<basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
            FullCourTetto, The22Columns, The22Beams, The22Hwall, 
            The11Frieze, TopTrellis >;
***        

In [140]:
def Trellis(args):
    H,L=args
    return COMP([R([2,3])(PI/2),
                EMBED(1),
                 MKPOL])([[[0,0],[L/3,0],
                           [2*L/3,0],
                           [L,0],[2*L/3,H/3],[L/3,2*H/3],[0,H]],
                          [[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,1],[1,6],
                          [6,2],[2,5],[5,3]],
                          [1,2,3,4,5,6,7,8,9,10,11]])

In [141]:
def Radial(n):
    def Radial_0(obj):
        return COMP([STRUCT,NN(n/3)])([COMP([STRUCT,NN(n/3)])
                                       ([obj,R([1,2])(2*PI/n)]), R([1,2])(2*PI/3)])
    return Radial_0

In [142]:
def TrellisTop3D():
    return COMP([T(2)(-0.25),
                S(2)(0.5),
                OFFSET([1,1,1]),
                Trellis])([hCol,r0-1])

In [143]:
TheTopTrellis = T(3)(3.5*hCol)(TrellisTop3D())

In [144]:
VIEW(TheTopTrellis)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f2015d0> >

### Image TheTopTrellis

![](img/TheTopTrellis.png)

In [145]:
TopTrellis = Radial(12)(TheTopTrellis)

In [146]:
VIEW(TopTrellis)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f201780> >

### Image TopTrellis

![](img/radial_Trellis.png)

In [147]:
top_trellis = STRUCT([The11Frieze,TopTrellis])
VIEW(top_trellis)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f201960> >

### Image Toptrellis

![](img/top_trellis.png)

In [148]:
VIEW(STRUCT([basement(),T(3)(hbasament),FullExtWall(),MedWallArcs(),CrossWalls(),FullTetto(),CrossCourtWall(),FullTopCourtWall(),FullCourTetto(),
             The22Columns,The22Beams,The22Hwall,
             The11Frieze,TopTrellis]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f201db0> >

![](img/struct_TopTrellis.png)

## Tralaccio Tetto intermedio

***
    DEF TrellisMed3D = (T:2:-0.25 ~ S:2:0.5 ~ OffSet:<1,1,1> ~ Trellis):<hcol,r2 - r0>;
    DEF TheMedTrellis = T:3:(hcol+3):TrellisMed3D;
    DEF MedTrellis = Radial:24:(T:1:r0:TheMedTrellis);
    DEF DoubleMedTrellis = STRUCT:<MedTrellis, R:<1,2>:(PI/24), MedTrellis>;
***

***
    DEF out = (CenteredCameras ~ STRUCT):
      <basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
            FullCourTetto, The22Columns, The22Beams, The22Hwall, 
            The11Frieze, TopTrellis , DoubleMedTrellis >;
***

***
    DEF out = STRUCT:
      <basement, T:3:hbasament, FullExtWall, FullTetto, CrossCourtWall,
            CrossWalls, MedWallArcs, TheMedColumns, FullTopCourtWall, 
            FullCourTetto, The22Columns, The22Beams, The22Hwall, 
            The11Frieze, TopTrellis , DoubleMedTrellis >;
***

In [149]:
def TrellisMed3D():
    return COMP([T(2)(-0.25),S(2)(0.5),OFFSET([1,1,1]),
                 Trellis])([hCol,r2-r0])

In [150]:
TheMedTrellis = T(3)(hCol+3)(TrellisMed3D())

In [151]:
MedTrellis = Radial(24)(T(1)(r0)(TheMedTrellis))

In [152]:
def DoubleMedTrellis():
    return STRUCT([MedTrellis,R([1,2])(PI/24),MedTrellis])

In [153]:
VIEW(DoubleMedTrellis())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f217180> >

### Image DoubleMedTrellis

![](img/doublemedtrellis.png)

In [154]:
def out():
    return STRUCT([basement(),T(3)(hbasament),FullExtWall(),FullTetto(),
                   CrossCourtWall(),CrossWalls(),MedWallArcs(),TheMedColumns(),
                   FullTopCourtWall(),FullCourTetto(),The22Columns,The22Beams,
                   The22Hwall,The11Frieze,TopTrellis,DoubleMedTrellis()])

In [155]:
VIEW(out())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f217720> >

![](img/ssr1.png)